<a href="https://colab.research.google.com/github/malakhovs04/ml_labs/blob/main/Laba_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack
from sklearn.linear_model import Ridge

1. Загрузите данные

In [8]:
data_train = pd.read_csv('/content/salary-train.csv')
data_test = pd.read_csv('/content/salary-test-mini.csv')

In [11]:
data_train

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London k ...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355
...,...,...,...,...
59995,As a result of continued growth First Class S...,Whitley Bay,contract,26400
59996,PHP MVC Web Developer MacclesfieldCirca ...,Macclesfield,permanent,26000
59997,Staff Nurse Nursing Home Baldock White Recru...,Baldock,NaN,24500
59998,This is one of the best agency side opportunit...,The City,permanent,65000


In [12]:
data_test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


2. Проведите предобработку

In [26]:
data_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True,inplace=True)
data_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True,inplace=True)

2.1. Преобразование текстов в векторы признаков

In [44]:
vectorize = TfidfVectorizer(min_df=5)
FullDescription_train = vectorize.fit_transform(data_train['FullDescription'])
FullDescription_test = vectorize.transform(data_test['FullDescription'])

2.2. Замена пропусков

In [32]:
data_train['LocationNormalized'].fillna('nan', inplace = True)
data_train['ContractTime'].fillna('nan', inplace = True)

<ipython-input-32-5b9194a92893>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_train['LocationNormalized'].fillna('nan', inplace = True)
<ipython-input-32-5b9194a92893>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

2.3. Примение DictVectorizer

In [41]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

2.4 Обьедините все полученные признаки в одну матрицу

In [46]:
joinedMatrix_train=hstack([FullDescription_train,X_train_categ])

3. Обучите гребневую регрессию

In [47]:
clf = Ridge(alpha=1, random_state=241)
clf.fit(joinedMatrix_train, data_train['SalaryNormalized'])

Ridge(alpha=1, random_state=241)

4. Постройте прогнозы

In [49]:
joinedMatrix_train = hstack([FullDescription_test, X_test_categ])
predict = clf.predict(joinedMatrix_train)
print(' '.join(map(lambda x: str(round(x, 2)), predict)))

56573.48 37194.91
